# 계층별 헤더 기반 문서 분할 테스트

이 노트북은 RAG 개발을 위한 계층별 문서 분할 전략을 테스트합니다.

## 목표
- 마크다운 문서를 헤더 계층별로 분할
- 각 계층(#, ##, ###, ####)별로 문서 청크 생성
- 기술 문서의 코드값, 메시지, 함수 규격 정보를 효과적으로 검색 가능하도록 최적화
- PNS purchaseState 질의 테스트를 통한 검증


In [12]:
# 필요한 라이브러리 import
import sys
import os
import json
from pathlib import Path

# code_dev_agent/16 디렉토리를 Python 경로에 추가
current_path = Path.cwd()
if current_path.name == '16':
    # 이미 code_dev_agent/16 디렉토리에 있는 경우
    sys.path.insert(0, str(current_path))
else:
    # 프로젝트 루트에서 실행하는 경우
    module_path = current_path / 'code_dev_agent' / '16'
    if module_path.exists():
        sys.path.insert(0, str(module_path))

from hierarchical_document_splitter import HierarchicalDocumentSplitter, DocumentChunk, demo_pns_query_test

print("라이브러리 로드 완료")


라이브러리 로드 완료


In [14]:
# 원본 문서 파일 경로 설정
document_path = "data/dev_center_guide_allmd_touched.md"

# 문서 읽기
with open(document_path, 'r', encoding='utf-8') as f:
    document_text = f.read()

print(f"문서 로드 완료: {len(document_text):,}자")
print(f"총 라인 수: {len(document_text.split())}")


문서 로드 완료: 383,563자
총 라인 수: 35227


In [15]:
# HierarchicalDocumentSplitter 초기화
splitter = HierarchicalDocumentSplitter(
    include_parent_context=True,  # 상위 컨텍스트 포함
    max_chunk_size=2000  # 최대 청크 크기
)

print("문서 분할기 초기화 완료")


문서 분할기 초기화 완료


In [16]:
# 문서 분할 실행
print("문서 분할 시작...")
chunks = splitter.split_document(document_text)
print(f"문서 분할 완료: {len(chunks)}개 청크 생성")


문서 분할 시작...
문서 분할 완료: 648개 청크 생성


In [24]:
cnt = 0

for chunk in chunks:
  if('PNS' in chunk.title or 'PNS' in chunk.content):
    print(f"idx: {cnt}")
    print(f"title: {chunk.title}")
    print(f"content: {chunk.content}")
    print(f"full_path: {chunk.full_path}")
    print(f"level: {chunk.level}")
    print(f"start_line: {chunk.start_line}")
    print(f"end_line: {chunk.end_line}")
    print(f"metadata: {chunk.metadata}")
    print("-" * 100)
    cnt += 1

print(f"PNS 관련 청크 수: {cnt}")

idx: 0
title: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
content: # 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.

보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.

{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}

{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요

* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}

If you are comfortable with English, please change the language to English from the upper left side in this page.

* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적용하기](v21/subs)
* [09. 원스토어 인앱결제 릴리즈 노트](v21/releasenote)
* [10. Sample App Download](v21/sample)
* [11. V21로 원스토어 인앱결제 업그레이드 하기](v21/upg

In [17]:
# 청크 요약 정보 출력
splitter.print_chunk_summary(chunks)


총 648개의 청크가 생성되었습니다.

레벨 1 헤더: 61개
  - 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 (1136자)
  - 01. 원스토어 인앱결제 개요 (18자)
  - 02. 사전준비 (10자)
  ... 및 58개 더

레벨 2 헤더: 100개
  - **원스토어 인앱결제란?** (385자)
  - **인앱 상품의 유형** (971자)
  - **결제 프로세스** (121자)
  ... 및 97개 더

레벨 3 헤더: 287개
  - 원스토어 로그인 하기 (94자)
  - **인앱 상품 구매 요청하기** (698자)
  - **구매 확인하기** (1707자)
  ... 및 284개 더

레벨 4 헤더: 170개
  - **GaaSignInClient 초기화** (239자)
  - **포그라운드 로그인하기** (309자)
  - **백그라운드 로그인하기** (296자)
  ... 및 167개 더



In [18]:
# PNS purchaseState 질의 테스트
demo_pns_query_test(chunks)


=== PNS purchaseState 질의 테스트 ===

'purchaseState' 관련 청크 31개 발견:

청크 1:
  제목: 구매 확인하기 (acknowledge)
  경로: 04. 원스토어 인앱결제 SDK를 사용해 구현하기 > 인앱 라이브러리 적용하기 > 구매 처리하기 > 구매 확인하기 (acknowledge)
  레벨: 4
청크 2:
  제목: 구매 확인하기 (acknowledge) (Part 1)
  경로: 04. 원스토어 인앱결제 SDK를 사용해 구현하기 > 인앱 라이브러리 적용하기 > 구매 처리하기 > 구매 확인하기 (acknowledge) > Part 1
  레벨: 5
청크 3:
  제목: 구매 확인하기 (acknowledge) (Part 2)
  경로: 04. 원스토어 인앱결제 SDK를 사용해 구현하기 > 인앱 라이브러리 적용하기 > 구매 처리하기 > 구매 확인하기 (acknowledge) > Part 2
  레벨: 5
청크 4:
  제목: 05. 원스토어 인앱결제 레퍼런스
  경로: 05. 원스토어 인앱결제 레퍼런스
  레벨: 1
청크 5:
  제목: 05. 원스토어 인앱결제 레퍼런스 (Part 2)
  경로: 05. 원스토어 인앱결제 레퍼런스 > Part 2
  레벨: 2
청크 6:
  제목: Annotations
  경로: Annotations
  레벨: 1
청크 7:
  제목: PurchaseData.PurchaseState
  경로: PurchaseData.PurchaseState
  레벨: 1
청크 8:
  제목: Constants
  경로: PurchaseData.PurchaseState > Constants
  레벨: 2
청크 9:
  제목: PURCHASED
  경로: PurchaseData.PurchaseState > Constants > PURCHASED
  레벨: 3
청크 10:
  제목: CANCEL
  경로: PurchaseData.PurchaseState > Constants > CANCEL
  레벨: 3
청

In [19]:
# 특정 PNS 청크의 상세 내용 확인
print("=== PNS 상세 정보 청크 내용 ===")

# PNS 상세 관련 청크 찾기
pns_detail_chunks = [chunk for chunk in chunks if "PNS 상세" in chunk.title or "Payment Notification" in chunk.title]

for chunk in pns_detail_chunks:
    print(f"\n제목: {chunk.title}")
    print(f"레벨: {chunk.level}")
    print(f"경로: {' > '.join(chunk.full_path)}")
    print("내용:")
    print("-" * 60)
    
    # purchaseState 관련 라인들만 추출하여 표시
    lines = chunk.content.split('\n')
    purchase_state_found = False
    
    for line in lines:
        if 'purchaseState' in line.lower():
            purchase_state_found = True
            print(f"★ {line.strip()}")
        elif purchase_state_found and ('COMPLETED' in line or 'CANCELED' in line):
            print(f"  → {line.strip()}")
    
    print()


=== PNS 상세 정보 청크 내용 ===

제목: 07. PNS(Payment Notification Service) 이용하기
레벨: 1
경로: 07. PNS(Payment Notification Service) 이용하기
내용:
------------------------------------------------------------


제목: **PNS 상세**
레벨: 2
경로: 07. PNS(Payment Notification Service) 이용하기 > **PNS 상세**
내용:
------------------------------------------------------------


제목: PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)
레벨: 3
경로: 07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)
내용:
------------------------------------------------------------


제목: PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버) (Part 1)
레벨: 4
경로: 07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버) > Part 1
내용:
------------------------------------------------------------


제목: PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버) (Part 2)
레벨: 4
경로: 07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > PNS Payment Notification 메시지

In [20]:
# 다양한 질의에 대한 테스트
print("=== 다양한 질의 테스트 ===")

test_queries = [
    "purchaseState",
    "결제 상태",
    "COMPLETED",
    "CANCELED", 
    "messageType",
    "구매 토큰",
    "purchaseToken",
    "개발자센터"
]

for query in test_queries:
    relevant = splitter.find_relevant_chunks(chunks, query)
    print(f"'{query}' 관련 청크: {len(relevant)}개")
    
    if relevant:
        print(f"  대표 청크: {relevant[0].title}")
        print(f"  경로: {' > '.join(relevant[0].full_path)}")
    print()


=== 다양한 질의 테스트 ===
'purchaseState' 관련 청크: 31개
  대표 청크: 구매 확인하기 (acknowledge)
  경로: 04. 원스토어 인앱결제 SDK를 사용해 구현하기 > 인앱 라이브러리 적용하기 > 구매 처리하기 > 구매 확인하기 (acknowledge)

'결제 상태' 관련 청크: 17개
  대표 청크: **개요**
  경로: 06. 원스토어 인앱결제 서버 API (API V7) > **개요**

'COMPLETED' 관련 청크: 34개
  대표 청크: 원스토어 서비스 설치하기
  경로: 04. 원스토어 인앱결제 SDK를 사용해 구현하기 > 인앱 라이브러리 적용하기 > 원스토어 서비스 설치하기

'CANCELED' 관련 청크: 6개
  대표 청크: USER\_CANCELED
  경로: PurchaseClient.ResponseCode > Constants > USER\_CANCELED

'messageType' 관련 청크: 5개
  대표 청크: PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)
  경로: 07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)

'구매 토큰' 관련 청크: 44개
  대표 청크: 구매 요청하기
  경로: 04. 원스토어 인앱결제 SDK를 사용해 구현하기 > 인앱 라이브러리 적용하기 > 구매 요청하기

'purchaseToken' 관련 청크: 80개
  대표 청크: 업그레이드 또는 다운그레이드
  경로: 04. 원스토어 인앱결제 SDK를 사용해 구현하기 > 인앱 라이브러리 적용하기 > 정기 결제 > 업그레이드 또는 다운그레이드

'개발자센터' 관련 청크: 20개
  대표 청크: setBase64PublicKey
  경로: PurchaseClient.Builder > Public Methods > setBase64Publi

## 결론 및 분석

이 테스트를 통해 다음을 확인할 수 있습니다:

1. **계층별 분할의 효과성**: 각 헤더 레벨별로 적절히 문서가 분할되는지
2. **질의 응답 품질**: "PNS의 purchaseState는 어떤 값이 있나요?" 같은 질의에 대한 답변 품질
3. **청크 크기 최적화**: RAG에 적합한 크기로 청크가 생성되는지
4. **컨텍스트 보존**: 상위 계층의 컨텍스트가 적절히 보존되는지

### 개선사항
- 필요시 청크 크기 조정
- 특정 패턴(테이블, 코드 블록 등)에 대한 특별 처리
- 메타데이터 활용 최적화

### 예상 결과
"PNS의 purchaseState는 어떤 값이 있나요?" 질의에 대해:
- **답변**: "COMPLETED : 결제완료 / CANCELED : 취소"
- 관련 청크가 적절히 식별되고 정확한 답변이 가능해야 함
